In [1]:
import numpy as np 
import pandas as pd 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import class_weight
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
import tensorflow as tf

In [2]:
# Load Data
mit_train_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_train.csv',header=None)
mit_test_data = pd.read_csv('/kaggle/input/heartbeat/mitbih_test.csv', header=None)

In [3]:
mit_train_data.shape

(87554, 188)

In [4]:
mit_test_data.shape

(21892, 188)

In [5]:
def bin_labels(index,labels):
    new_labels = np.copy(labels)
    new_labels[new_labels==index]=10
    new_labels[new_labels!=10]=0
    new_labels[new_labels==10]=1
    return new_labels

In [6]:
def pre_process(index,data,shuffle=True):
    X_train = np.asarray(data.drop(columns=[187]))
    y_train = np.asarray(data[187])
    y_train = bin_labels(index,y_train)
    if shuffle:
        indexes = np.arange(int(X_train.shape[0]))
        indexes = np.random.RandomState(seed=42).permutation(indexes)  # shuffle data to randomly select
        X_train_new = X_train[indexes]
        y_train_new= y_train[indexes]
    else:
        X_train_new = X_train
        y_train_new = y_train
    y_train_new = y_train_new.astype(int)
    y_train_new_cat = to_categorical(y_train_new)
    return X_train_new,y_train_new,y_train_new_cat

# Build Model

In [7]:
from keras import layers
from keras.layers import Dense, Dropout, BatchNormalization, Flatten
from keras.layers import Conv1D, GlobalAveragePooling1D, AveragePooling1D
from keras.models import Sequential,save_model,load_model
from keras import optimizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf

In [8]:
def build_model():
    model = Sequential()
    
    # Convolutional block 1
    model.add(Conv1D(256,2,strides=1,activation = 'relu',input_shape=(187,1)))
    model.add(Conv1D(256,2,strides=1,activation = 'relu'))
    #model.add(Conv1D(256,2,strides=1,activation = 'relu'))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    # Convolutional block 2
    model.add(Conv1D(256,2,strides=1,activation = 'relu'))
    model.add(Conv1D(256,2,strides=1,activation = 'relu'))
    #model.add(Conv1D(256,2,strides=1,activation = 'relu'))

    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    
    model.add(GlobalAveragePooling1D())
              
    model.add(Dense(128,activation = 'relu'))
    
    model.add(Dense(2, activation='softmax'))    # now a binary classification problem      
    
    return model

In [9]:
# Fine tunes the learning after the 80th epoch
def lr_decay(epoch):
    if epoch < 80:
        learn_rate = 0.001
    else: 
        learn_rate = 0.0001
    return learn_rate

In [10]:
""" Define focal loss function as per 
    https://www.dlology.com/blog/
    multi-class-classification-with-focal-loss-for-imbalanced-datasets/""" 
def focal_loss_fnc(gamma,alpha):
    
    gamma = float(gamma)
    alpha = float(alpha)
    
    def focal_loss_fixed(y_true, y_pred):
        """Focal loss for multi-classification
        FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
        Notice: y_pred is probability after softmax
        gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
        d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
        Focal Loss for Dense Object Detection
        https://arxiv.org/abs/1708.02002

        Arguments:
            y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

        Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
        """
        epsilon = 1.e-9
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)

        model_out = tf.add(y_pred, epsilon)
        ce = tf.multiply(y_true, -tf.math.log(model_out))
        weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
        fl = tf.multiply(alpha, tf.multiply(weight, ce))
        reduced_fl = tf.reduce_max(fl, axis=1)
        return tf.reduce_mean(reduced_fl)
    return focal_loss_fixed

In [11]:
def train_model(model,X_train,y_train,X_valid,y_valid,filename,n_epochs=100,batch_size=400,
                learn_rate=0.001,focal_loss=True,gamma=2,alpha=0.25):
    
    if focal_loss:
        loss = focal_loss_fnc(gamma,alpha) # still need to define
        
    else:
        loss = 'categorical_crossentropy'
    
    adam = optimizers.Adam(learning_rate=learn_rate)
    
    # may want to change metrics later
    model.compile(optimizer=adam,loss=loss,metrics=['accuracy'])
    
    # Reduce learning rate to 0.0001 after 80th epoch
    callbacks = LearningRateScheduler(lr_decay)
        
    history = model.fit(X_train,y_train,validation_data=(X_valid,y_valid), epochs=n_epochs,batch_size=batch_size,callbacks=callbacks)
    
    print("\nEvaluating...", flush=True)
    print('Training data:', flush=True)
    loss, acc = model.evaluate(X_train, y_train, verbose=1)
    print("  Training : loss %.3f - acc %.3f" % (loss, acc))
    
    print('Saving model...', flush=True)
    save_model(
    model,
    filename,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
    )
    
    return model, history

# 0 vs rest

In [12]:
# X_train0, labels0, y_train0 = pre_process(0,mit_train_data)
# X_train_new0 = X_train0.reshape(87554,187,1)

# X_test0, test_labels0, y_test0 = pre_process(0,mit_test_data,shuffle=False)
# X_test_new0 = X_test0.reshape(X_test0.shape[0],187,1)

In [13]:
# filename = "model_0vR"
# model_0vR,history_0vR = train_model(build_model(),
#                                     X_train_new0,
#                                     y_train0,
#                                     X_test_new0,
#                                     y_test0,
#                                     filename,
#                                     focal_loss=True)
# hist_df_0vR = pd.DataFrame(history_0vR.history)
# hist_df_0vR.to_csv("history_0vR.csv", index=False)

# 1 vs rest

In [14]:
X_train1, labels1, y_train1 = pre_process(1,mit_train_data)
X_train_new1 = X_train1.reshape(87554,187,1)

X_test1, test_labels1, y_test1 = pre_process(1,mit_test_data,shuffle=False)
X_test_new1 = X_test1.reshape(X_test1.shape[0],187,1)

In [15]:
test_labels1

array([0, 0, 0, ..., 0, 0, 0])

In [16]:
filename = "model_1vR"
model_1vR,history_1vR = train_model(build_model(),
                                    X_train_new1,
                                    y_train1,
                                    X_test_new1,
                                    y_test1,
                                    filename,
                                    focal_loss=True)
hist_df_1vR = pd.DataFrame(history_1vR.history)
hist_df_1vR.to_csv("history_1vR.csv", index=False)

Epoch 1/100
219/219 [==============================] - 14s 64ms/step - loss: 0.0081 - accuracy: 0.9740 - val_loss: 0.0096 - val_accuracy: 0.9746
Epoch 2/100
219/219 [==============================] - 13s 61ms/step - loss: 0.0053 - accuracy: 0.9810 - val_loss: 0.0114 - val_accuracy: 0.9746
Epoch 3/100
219/219 [==============================] - 13s 61ms/step - loss: 0.0047 - accuracy: 0.9831 - val_loss: 0.0080 - val_accuracy: 0.9819
Epoch 4/100
219/219 [==============================] - 14s 62ms/step - loss: 0.0046 - accuracy: 0.9839 - val_loss: 0.0057 - val_accuracy: 0.9821
Epoch 5/100
219/219 [==============================] - 13s 61ms/step - loss: 0.0043 - accuracy: 0.9846 - val_loss: 0.0124 - val_accuracy: 0.9699
Epoch 6/100
219/219 [==============================] - 13s 61ms/step - loss: 0.0042 - accuracy: 0.9849 - val_loss: 0.0066 - val_accuracy: 0.9829
Epoch 7/100
219/219 [==============================] - 13s 61ms/step - loss: 0.0041 - accuracy: 0.9852 - val_loss: 0.0055 - val_ac